## **MODEL EĞİTİMİ**

In [3]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

**BATCH_SIZE**: Her 128 örnekte bir adım atmak demektir.Yani ilk 128 sample alınacak traininge sokulacak ardından 129-256 arası alınacak. Burada karşılaşılan en önemli sorun son sette belirtilen kadar sample kalamaması.Bu problem şu şekilde çözülür;son set kaç tane kalacak ise ayrı olarak eğitilir.

**NUM_CLASSES:** 10 tane sınıfımız var.digits(0-1-2-3-4-5-6-7-8-9)

**EPOCHS:** Tüm veri setini 12 kere yineliyoruz.
(One epoch is too big to feed to the computer at once. So, we divide it in several smaller batches. We use more than one epoch because passing the entire dataset through a neural network is not enough and we need to pass the full dataset multiple times to the same neural network.)

In [4]:
batch_size = 128
num_classes = 10
epochs = 12

Görüntü boyutları

In [5]:
img_rows, img_cols = 28, 28

Datayı alırken test ve train olarak ayırıp bünyemize katıyoruz

In [7]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Kullanılan Backend e göre değişiklik yapıyoruz.
Yani data formatı konrol ediyoruz.

     keras.backend.image_data_format()  
     #ile bakabiliriz

Görüntü verilerini üç boyulu bir şekilde temsil etmenin iki yolu vardır, channels_first ve channels_last :

**Channels Last:** Image data is represented in a three-dimensional array where the last channel represents the color channels, 

*e.g. [rows][cols][channels].*

**Channels First:** Image data is represented in a three-dimensional array where the first channel represents the color channels, 

*e.g. [channels][rows][cols].*

In [ ]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

Sınıf vektörleri ikili-binary- formuna dönüştürülüyor.

(Convert Class Vector to Binary Class Matrices)

to_categorical():one-hot-encoding uygulanır.

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Hazırlayacağımız model Keras'ın Sequeltial modeli. Sıralı katmanlardan oluşan bir model. Yeni bir sequential(ardışık) model objesi yaratalım,ismi de model olsun. Katmanları eklemeye başlayalım. Bunun için add() fonksiyonunu kullanacağız.

In [ ]:
model = Sequential()

3x3 boyutunda 32 adet filtreden oluşan ReLu aktivasyonlu CONV katmanı
eklenmesi.

In [ ]:
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))

3x3 boyutunda 64 adet filtreden oluşan ReLu aktivasyonlu CONV katmanı
eklenmesi.

In [1]:
model.add(Conv2D(64, (3, 3), activation='relu'))

NameError: name 'model' is not defined

2x2 boyutlu çerçeveden oluşan MAXPOOL katmanı eklenmesi

In [ ]:
model.add(MaxPooling2D(pool_size=(2, 2)))

Her seferinde nöronların %25 i atılsın.

Weightlerin %25ini düşüren bir Droput katmanı ekliyoruz. Droput katmanı,Neural Networklerde overfiring i engellemek için icat edilmiş bir yöntemdir.

In [ ]:
model.add(Dropout(0.25))

Tam bağlantılı(fully connected)katmanına geçiş olacağı için düzleştirme yapılıyor.

In [ ]:
model.add(Flatten())

128 nörondan oluşan ReLu aktivasyonlu FC katmanı ekleniyor

In [ ]:
model.add(Dense(128, activation='relu'))

Her seferinde yarısını atalım nöronların

In [ ]:
model.add(Dropout(0.5))

Çıkış katmanına sınıf sayısı kadar (10) Softmax aktivasyonlu nöron ekleyelim

In [ ]:
model.add(Dense(num_classes, activation='softmax'))

Adadelta optimizasyon yöntemi ve cross entropy yitim (loss) fonksiyonunun kullanımı

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

Eğitim işlemi

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Test işlemi ve sonuçların ekrana yansıtılması

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

    Test loss: 0.02935905354363167
    Test accuracy: 0.9908

Her defasında modeli eğitmek zorunda kalmamak için eğitilmiş modelin kaydedilmesi.

In [ ]:
model.save('mnistCNN.h5')


In [ ]:
model.summary()

Bu kısımda eğitim doğruluğu ve validation accuracy eğrileri grafkte gösterilmiştir.

In [ ]:
from keras.models import load_model
import matplotlib.pyplot as plt
model = load_model('mnistCNN.h5')

hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.3, callbacks=my_callbacks)

epoch_list = list(range(1, len(hist.history['acc']) + 1))
plt.plot(epoch_list, hist.history['acc'], epoch_list, hist.history['val_acc'])
plt.legend(("Training Accuracy", "Validation Accuracy"))
plt.show()



## **MODELİMİZİ OPENCV İLE ÖRNEKLERDE DENEME**


Modelimizi tekrar eğitip zaman kaybetmemek için eğitilmiş modeli load_model() metodu ile yüklüyoruz.

In [ ]:

import cv2
import numpy as np
from keras.models import load_model

model = load_model('mnistCNN.h5')

In [ ]:
def get_numbers(y_pred):
    for number, per in enumerate(y_pred[0]):
        if per != 0:
            final_number = str(int(number))
            per = round((per * 100), 2)
            return final_number, per

Sample'ımızı yüklüyoruz ve bazı morfolojik işlemlere tabi tutuyoruz.

In [ ]:
img=cv2.imread("MNISTsamples.png")


img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
img_gau = cv2.GaussianBlur(img_gray, (5, 5), 0)
ret, thresh = cv2.threshold(img_gau, 80, 255, cv2.THRESH_BINARY_INV)
#cv2.imshow("Frame thersh",thresh)


**Görüntülerin erozyonu ve genişlemesi**(Erosion and Dilation of images)

Erosion:Ön plan nesnesinin sınırlarını aşındırır. Bir görüntünün özelliklerini azaltmak için kullanılır.

*Basics of Erosion:*
1. Erodes away the boundaries of foreground object
2. Used to diminish the features of an image.

*Working of erosion:*

1. A kernel(a matrix of odd size(3,5,7) is convolved with the image.
2. A pixel in the original image (either 1 or 0) will be considered 1 only if all the pixels under the kernel is 1, otherwise it is eroded (made to zero).
3. Thus all the pixels near boundary will be discarded depending upon the size of kernel.
4. So the thickness or size of the foreground object decreases or simply white region decreases in the image.

 *Küçük beyaz gürültüleri gidermek için kullanışlıdır.
 
 *İki bağlı nesneyi ayırmak için kullanılır.


Dilation:Nesne alanını arttırır.
Özellikleri vurgulamak için kullanılır.

*Basics of dilation:*
1. Increases the object area
2. Used to accentuate features

*Working of dilation:*
1. A kernel(a matrix of odd size(3,5,7) is convolved with the image
2. A pixel element in the original image is ‘1’ if atleast one pixel under the kernel is ‘1’.
3. It increases the white region in the image or size of foreground object increases

 *Gürültünün giderilmesi durumunda erozyonu dilation izler.Çünkü erozyon beyaz gürültüleri engellerken aynı zamanda nesnemizi daraltır. Bu yüzden onu genişletmeliyiz. Gürültü kaybolduğundan geri gelmeyecektir ancak nesnemizin alanı artacak bu sayede.

In [ ]:
kernel = np.ones((5, 5), np.uint8)
dilation = cv2.dilate(thresh, kernel, iterations=1)
#cv2.imshow("Frame dilation", thresh)

In [ ]:
edged = cv2.Canny(dilation, 50, 250)
#cv2.imshow("Frame edged", thresh)

In [ ]:
cv2.imwrite('capture.jpg',img)
capture_img = cv2.imread('capture.jpg')

img2 = capture_img.copy()
img_gray = cv2.cvtColor(capture_img, cv2.COLOR_RGB2GRAY)
img_gau = cv2.GaussianBlur(img_gray, (5, 5), 0)
ret, thresh = cv2.threshold(img_gau, 80, 255, cv2.THRESH_BINARY_INV)

In [ ]:
kernel = np.ones((5, 5), np.uint8)
dilation = cv2.dilate(thresh, kernel, iterations=1)

In [ ]:
edged = cv2.Canny(dilation, 50, 250)

contours, hierachy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


In [ ]:
num_str = ''
per = ''

num_list = []

Sayıyı tespit halinde etrafına dikdörtgen çizmemizi sağlar ve sol üst köşesine tahmimi rakamı yazar.

In [ ]:
if len(contours) > 0:
    for c in contours:
        if cv2.contourArea(c) > 2500:
            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(img2, (x, y), (x + w, y + h), (255, 0, 255), 3)

            new_img = thresh[y:y + h, x:x + w]
            new_img2 = cv2.resize(new_img, (28, 28))
            im2arr = np.array(new_img2)
            im2arr = im2arr.reshape(1, 28, 28, 1)
            y_pred = model.predict(im2arr)

            num, per = get_numbers(y_pred)
            num_list.append(str(int(num)))
            num_str = '[' + str(str(int(num))) + ']'
            cv2.putText(img2, num_str, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2, cv2.LINE_AA)


str1 burada tahmin edilen sayılar

In [ ]:
str1 = ' '.join(num_list)

Tahmini sayıları frame in sol üstüne yazdıran kısım.

In [ ]:
if (str1 != ''):
    y_p = str('Predicted Value is ' + str(str1))
    print(y_p)
    cv2.putText(img2, y_p, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)


In [ ]:
cv2.imshow("Capture Frame", img2)
#cv2.imshow("Contours Frame", thresh)

In [ ]:
cv2.waitKey(0)

In [1]:
def f1():
    n = 1000000
    if n != 0:
        n -= 1

In [3]:
%timeit f1()

224 ns ± 38.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
